#  DSC478 - Programming Machine Learning Applications
## Assignment 4 - Lavinia Wang

### Problem 2. Item-Based Joke Recommendation

Use a modified version of the item-based recommender algorithm from Ch.14 of Machine Learning in Action and use it on joke ratings data based on Jester Online Joke Recommender System. The modified version of the code is provided in the module itemBasedRec.py. Most of the module will be used as is, but you will add some additional functionality. The data set contains two files. The file "modified_jester_data.csv" contains the ratings on 100 jokes by 1000 users (each row is a user profile). The ratings have been normalized to be between 1 and 21 (a 20-point scale), with 1 being the lowest rating. A zero indicated a missing rating. The file "jokes.csv" contains the joke ids mapped to the actual text of the jokes.

Dataset: <a href='http://facweb.cs.depaul.edu/mobasher/classes/CSC478/Data/jokes.zip'>jokes.zip</a> Dataset description <a href='http://eigentaste.berkeley.edu/dataset/'>here</a>

In [1]:
# Import modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import os 

from numpy import *
from numpy import linalg as la

In [2]:
# Change working directory
os.chdir('/resources/CSC478/Assignment4')

#### a. Load in the joke ratings data and the joke text data into appropriate data structures.

In [3]:
ratings = np.genfromtxt('modified_jester_data.csv', delimiter=',')
ratings.shape

(1000, 100)

In [4]:
jokes = np.genfromtxt('jokes.csv',delimiter=',',dtype=str)
jokes = np.array(jokes[:,1])

In [5]:
jokes[0:5]

array(['A man visits the doctor. The doctor says "I have bad news for you.You have cancer and Alzheimer\'s disease". The man replies "Well thank God I don\'t have cancer!"',
       'This couple had an excellent relationship going until one day he came home from work to find his girlfriend packing. He asked her why she was leaving him and she told him that she had heard awful things about him. "What could they possibly have said to make you move out?" "They told me that you were a pedophile." He replied "That\'s an awfully big word for a ten year old."',
       "Q. What's 200 feet long and has 4 teeth? A. The front row at a Willie Nelson Concert.",
       "Q. What's the difference between a man and a toilet? A. A toilet doesn't follow you around after you use it.",
       "Q. What's O. J. Simpson's Internet address? A.\tSlash slash backslash slash slash escape."],
      dtype='<U1198')

In [6]:
jokes.shape

(100,)

In [7]:
int(0.2*10)

2

#### b. Complete the definition for the function "test". This function iterates over all users and for each performs evaluation (by calling the provided "cross_validate_user" function), and returns the error information necessary to compute Mean Absolute Error (MAE). Use this function to perform evaluation (with 20% testratio for each user) comparing MAE results using standard item-based collaborative filtering (based on the rating prediction function "standEst") with results using the SVD-based version of the rating item-based CF (using "svdEst" as the prediction engine). [Note: See comments provided in the module for hints on accomplishing these tasks.]

In [8]:
from numpy import *
from numpy import linalg as la
import numpy as np

def ecludSim(inA,inB):
    return 1.0 / (1.0 + la.norm(inA - inB))

def pearsSim(inA,inB):
    if len(inA) < 3 : return 1.0
    return 0.5 + 0.5 * corrcoef(inA, inB, rowvar = 0)[0][1]

def cosSim(inA,inB):
    num = float(inA.T * inB)
    denom = la.norm(inA)*la.norm(inB)
    return 0.5 + 0.5 * (num / denom)

def standEst(dataMat, user, simMeas, item):
    n = shape(dataMat)[1]
    simTotal = 0.0; ratSimTotal = 0.0
    for j in range(n):
        userRating = dataMat[user,j]
        if userRating == 0: continue
        overLap = nonzero(logical_and(dataMat[:,item]>0, \
                                      dataMat[:,j]>0))[0]
        if len(overLap) == 0: similarity = 0
        else: similarity = simMeas(dataMat[overLap,item], \
                                   dataMat[overLap,j])
        #print 'the %d and %d similarity is: %f' % (item, j, similarity)
        simTotal += similarity
        ratSimTotal += similarity * userRating
    if simTotal == 0: return 0
    else: return ratSimTotal/simTotal
    
def svdEst(dataMat, user, simMeas, item):
    n = shape(dataMat)[1]
    simTotal = 0.0; ratSimTotal = 0.0
    data=mat(dataMat)
    U,Sigma,VT = la.svd(data)
    Sig4 = mat(eye(4)*Sigma[:4]) #arrange Sig4 into a diagonal matrix
    xformedItems = data.T * U[:,:4] * Sig4.I  #create transformed items
    for j in range(n):
        userRating = data[user,j]
        if userRating == 0 or j==item: continue
        similarity = simMeas(xformedItems[item,:].T,\
                             xformedItems[j,:].T)
        #print 'the %d and %d similarity is: %f' % (item, j, similarity)
        simTotal += similarity
        ratSimTotal += similarity * userRating
    if simTotal == 0: return 0
    else: return ratSimTotal/simTotal

# This function is not needed for Assignment 4, but may be useful for experimentation
def recommend(dataMat, user, N=3, simMeas=cosSim, estMethod=standEst):
    unratedItems = nonzero(dataMat[user,:].A==0)[1] #find unrated items 
    if len(unratedItems) == 0: return 'you rated everything'
    itemScores = []
    for item in unratedItems:
        estimatedScore = estMethod(dataMat, user, simMeas, item)
        itemScores.append((item, estimatedScore))
    return sorted(itemScores, key=lambda jj: jj[1], reverse=True)[:N]

# This function performs cross-validation on a single user based on the test_ratio
# For example, with test_ratio = 0.2, 5-fold x-validation is performed where in each fold, 
# 20 percent of rated items are withheld and the rest are used to estimate the withheld ratings

def cross_validate_user(dataMat, user, test_ratio, estMethod=standEst, simMeas=pearsSim):
    number_of_items = np.shape(dataMat)[1]
    rated_items_by_user = np.array([i for i in range(number_of_items) if dataMat[user,i]>0])
    test_size = int(test_ratio * len(rated_items_by_user))
    test_indices = np.random.randint(0, len(rated_items_by_user), test_size)
    withheld_items = rated_items_by_user[test_indices]
    original_user_profile = np.copy(dataMat[user])
    dataMat[user, withheld_items] = 0 # So that the withheld test items is not used in the rating estimation below
    error_u = 0.0
    count_u = len(withheld_items)

    # Compute absolute error for user u over all test items
    for item in withheld_items:
        # Estimate rating on the withheld item
        estimatedScore = estMethod(dataMat, user, simMeas, item)
        error_u = error_u + abs(estimatedScore - original_user_profile[item])	
    
    # Now restore ratings of the withheld items to the user profile
    for item in withheld_items:
        dataMat[user, item] = original_user_profile[item]
        
    # Return sum of absolute errors and the count of test cases for this user
    # Note that these will have to be accumulated for each user to compute MAE
    return error_u, count_u

In [9]:
def test(dataMat, test_ratio, estMethod):
    # Write this function to iterate over all users and for each perform cross-validation on items by calling
    # the above cross-validation function on each user.
    # MAE will be the ratio of total error across all test cases to the total number of test cases, for all users
    
    #row = dataMat[0,]
    #print "A row is: ", row
    total_error=0.0
    total_count=0
    for user in range(dataMat.shape[0]):
        error, count = cross_validate_user(dataMat, user, .2, estMethod)
        total_error = total_error + error
        total_count = total_count + count
    MAE = total_error/total_count
    print ('Mean Absoloute Error for ',estMethod,' : ', MAE)
    return MAE

In [10]:
def kNearestNeighbors(inX, dataSet, k, measure):
    distances = []
    for i in range(dataSet.shape[1]):
        vector = dataSet[:,i]
        distance = measure(inX, vector)
        distances.append(distance)
    #print "Before, distances is: ", distances
    distancesArray = np.array(distances)
    #print "After, distances is: ", distancesArray
    sortedDistIndicies = distancesArray.argsort()
    kNeighbors = zeros((k,dataSet.shape[1]))
    topIndicies = []
    classCount={}
    for i in range(k):
        #voteIlabel = labels[sortedDistIndicies[i]]
        kNeighbors[i,:] = dataSet[sortedDistIndicies[i],:]
        topIndicies.append(sortedDistIndicies[i])
        #print "sortedDistIndices: is", sortedDistIndicies
        #print "kNeighbors is: ", kNeighbors
        #classCount[voteIlabel] = classCount.get(voteIlabel,0) + 1
    #sortedClassCount = sorted(classCount.iteritems(),key=operator.itemgetter(1), reverse=True)
    return kNeighbors, topIndicies  #, sortedClassCount[0][0]

def load_jokes(file):
    jokes = genfromtxt(file, delimiter=',', dtype=str)
    jokes = np.array(jokes[:,1])
    return jokes

def get_joke_text(jokes, id):
    return jokes[id]

# dataMat = genfromtxt('modified_jester_data.csv',delimiter=',')

# test(dataMat, 0.2, svdEst)
# test(dataMat, 0.2, standEst)

# jokes = load_jokes('jokes.csv')
# print_most_similar_jokes(dataMat, jokes, 3, 5, pearsSim)

''' See example output below:

Selected joke: 

Q. What's the difference between a man and a toilet? A. A toilet doesn't follow you around after you use it.

Top 5 Recommended jokes are :

Q: What's the difference between a Lawyer and a Plumber? A: A Plumber works to unclog the system. 
_______________
What do you call an American in the finals of the world cup? "Hey Beer Man!" 
_______________
Q. What's 200 feet long and has 4 teeth? <P>A. The front row at a Willie Nelson Concert. 
_______________
A country guy goes into a city bar that has a dress code and the maitred' demands he wear a tie. Discouraged the guy goes to his car to sulk when inspiration strikes: He's got jumper cables in the trunk! So he wrapsthem around his neck sort of like a string tie (a bulky string tie to be sure) and returns to the bar. The maitre d' is reluctant but says to the guy "Okay you're a pretty resourceful fellow you can come in... but just don't start anything"!   
_______________
What do you get when you run over a parakeet with a lawnmower? <P>Shredded tweet. 
_______________

'''

' See example output below:\n\nSelected joke: \n\nQ. What\'s the difference between a man and a toilet? A. A toilet doesn\'t follow you around after you use it.\n\nTop 5 Recommended jokes are :\n\nQ: What\'s the difference between a Lawyer and a Plumber? A: A Plumber works to unclog the system. \n_______________\nWhat do you call an American in the finals of the world cup? "Hey Beer Man!" \n_______________\nQ. What\'s 200 feet long and has 4 teeth? <P>A. The front row at a Willie Nelson Concert. \n_______________\nA country guy goes into a city bar that has a dress code and the maitred\' demands he wear a tie. Discouraged the guy goes to his car to sulk when inspiration strikes: He\'s got jumper cables in the trunk! So he wrapsthem around his neck sort of like a string tie (a bulky string tie to be sure) and returns to the bar. The maitre d\' is reluctant but says to the guy "Okay you\'re a pretty resourceful fellow you can come in... but just don\'t start anything"!   \n______________

In [11]:
%%time
test(ratings, .2, standEst)

Mean Absoloute Error for  <function standEst at 0x3fff68d51a60>  :  3.6936801967915636
CPU times: user 1min 51s, sys: 4 ms, total: 1min 51s
Wall time: 1min 51s


3.6936801967915636

In [12]:
%%time
test(ratings,.2, svdEst)

Mean Absoloute Error for  <function svdEst at 0x3fff68d516a8>  :  3.612726394645625
CPU times: user 21min 47s, sys: 20 ms, total: 21min 47s
Wall time: 21min 47s


3.612726394645625

#### c. Write a new function "print_most_similar_jokes" which takes the joke ratings data, a query joke id, a parameter k for the number of nearest neighbors, and a similarity metric function, and prints the text of the query joke as well as the texts of the top k most similar jokes based on user ratings. [Note: For hints on how to accomplish this task, please see comments at the end of the provided module as well as comments for the provided stub function.]

In [13]:
def print_most_similar_jokes(dataMat, jokes, queryJoke, k, metric=pearsSim):
    # Write this function to find the k most similar jokes (based on user ratings) to a queryJoke
    # The queryJoke is a joke id as given in the 'jokes.csv' file (an corresponding to the a column in dataMat)
    # You must compare ratings for the queryJoke (the column in dataMat corresponding to the joke), to all
    # other joke rating vectors and return the top k. Note that this is the same as performing KNN on the 
    # columns of dataMat. The function must retrieve the text of the joke from 'jokes.csv' file and print both
    # the queryJoke text as well as the text of the returned jokes.
    queryJokeVector = dataMat[:,queryJoke]
    print ("Selcted joke: ")
    print()
    print (jokes[queryJoke])
    print()
    dataMatArray = np.array(dataMat)
    knn, indicies = kNearestNeighbors(queryJokeVector, dataMat, k, metric)	
    print ("The top %d recommended jokes are: "%(k))
    for ind in indicies:
        jok = jokes[ind]
        print()
        print (jok)
    #print "Done"

In [14]:
print_most_similar_jokes(ratings, jokes, 3, 5, pearsSim)

Selcted joke: 

Q. What's the difference between a man and a toilet? A. A toilet doesn't follow you around after you use it.

The top 5 recommended jokes are: 

A dog walks into Western Union and asks the clerk to send a telegram. He fills out a form on which he writes down the telegram he wishes to send: "Bow wow wow Bow wow wow."The clerk says "You can add another 'Bow wow' for the same price."The dog responded "Now wouldn't that sound a little silly?"

Q:  What did the blind person say when given some matzah?A:  Who the hell wrote this?

Q. Did you hear about the dyslexic devil worshipper? A. He sold his soul to Santa.

Q. What is orange and sounds like a parrot?  A. A carrot.

A guy goes into confession and says to the priest "Father I'm 80 years old widower with 11 grandchildren. Last night I met two beautiful flight attendants. They took me home and I made love to both of them. Twice."The priest said: "Well my son when was the last time you were in confession?" "Never Father I'm 